Diff PRE-POST

In [84]:
from scipy.stats import gaussian_kde

def diff_size(nonPicked_ratio, all_measures, nonPicked_measures):
    picked_ratio = 1 - nonPicked_ratio

    # By miu,sigma , assume normalization
    miu_all = all_measures.mean()
    sigma_all = all_measures.std()
    miu_nonPicked = nonPicked_measures.mean()
    sigma_nonPicked = nonPicked_measures.std()
    miu_picked = (1 / picked_ratio) * miu_all - (nonPicked_ratio / picked_ratio) * miu_nonPicked
    sigma_picked = np.sqrt((1 / picked_ratio) ** 2 * sigma_all ** 2 - (nonPicked_ratio / picked_ratio) ** 2 * sigma_nonPicked ** 2)

    # By kde
    x_values = np.linspace(all_measures.min(), all_measures.max(), num=int(all_measures.max() - all_measures.min()))
    kde_all = gaussian_kde(all_measures)(x_values)
    kde_nonPicked = gaussian_kde(nonPicked_measures)(x_values)
    kde_picked = (1 / picked_ratio) * kde_all - (nonPicked_ratio / picked_ratio) * kde_nonPicked
    kde_miu = np.sum(kde_picked * x_values)
    kde_sigma = np.sqrt(np.sum(kde_picked * (x_values - kde_miu) ** 2))

    # By hist
    hist_all, bins, p = plt.hist(all_measures,density=True)
    hist_nonPicked = plt.hist(nonPicked_measures, density=True,bins=bins)[0]
    hist_picked = (1 / picked_ratio) * hist_all - (nonPicked_ratio / picked_ratio) * hist_nonPicked
    hist_picked = hist_picked/ np.sum(hist_picked)
    bins = [(var+ bins[i + 1])/2 for i, var in enumerate(bins) if i+1 != len(bins)]
    hist_miu = np.sum(hist_picked * bins)
    hist_sigma = np.sqrt(np.sum(hist_picked * (bins - hist_miu) ** 2))

    return (miu_picked, sigma_picked), (kde_miu, kde_sigma), (hist_miu, hist_sigma)

In [66]:
def map_tree_into_plot(row,T):
    df = pd.read_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\Mapping\Map_trees_to_plot.csv')
    t = df[(df['row']==int(row)) & (df['tree']==T)]
    return t.iloc[0]['plot']

Debug counter

In [67]:
import os
from vision.tools.manual_slicer import slice_to_trees
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [75]:
color_dict = {"bin1": [300, 15], "bin2": [15, 45], "bin3": [45, 90], "bin4": [90, 165]}


def filter_logic(path, analysis='counter'):
    df = pd.read_csv(os.path.join(path, 'measures.csv'))

    if analysis == 'size':
        df_res = df[(df['width'] < 90) & (df['height'] < 90)]
        # df_res = df[df.apply(lambda x: False if (x.width / x.height > 1.25) or (x.width / x.height < 0.75) else True, axis=1)]
    elif analysis == 'color':
        df_res = df[(df['color'] < 165) & (df['color'] > 300)]
    else:
        df_res = df
    return df_res

In [69]:
def trackers_into_value(df_tree, df_res, analysis='counter'):
    df_tree['end'].replace([-1], math.inf,inplace=True)
    plot_det = []
    const = 0
    for frame_id, df_frame in df_res.groupby('frame'):
        limit = df_tree[df_tree['frame_id'] == frame_id]
        if limit.empty:
            continue
        limit = limit.iloc[0]
        df = df_frame[(df_frame['x1'] + const > limit['start']) & (df_frame['x1'] < limit['end'] - const)]
        plot_det.append(df)
    if analysis == 'counter':
        var = len(pd.concat(plot_det,axis=0)['track_id'].unique())
    elif analysis == 'size':
        df_group = pd.concat(plot_det,axis=0).groupby('track_id')
        print(len(df_group))
        var = df_group.apply(lambda x : x.width.mean() if x.width.mean() > x.height.mean() else x.height.mean())
    return var

In [79]:
def apply_trees_counter(path, row):
    df = pd.DataFrame()
    # in case that no measures.csv or no json
    try:
        row_folder = os.path.join(path, row)
        df_res = filter_logic(row_folder)
        print(df_res)
        trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]), None, None)
    except:
        return df

    for tree_id, df_tree in trees_slices.groupby('tree_id'):
        count = trackers_into_value(df_tree, df_res, analysis='counter')
        plot_id = map_tree_into_plot(row, tree_id)
        df = df.append({'row': row, 'plot_id': plot_id, 'fruits_num': count}, ignore_index=True)
    return df

In [82]:
def run_counter(scan_pre, scan_post, output):
    df_pre,df_post = pd.DataFrame(),pd.DataFrame()
    for (pre,post) in zip(os.listdir(scan_pre),os.listdir(scan_post)):
        print(pre,post)
        df_pre = pd.concat([df_pre,apply_trees_counter(scan_pre,pre)],axis=0)
        df_post = pd.concat([df_post,apply_trees_counter(scan_post,post)],axis=0)
    df_sum = df_pre.merge(df_post,how='inner',on = ['row','plot_id'],suffixes=['pre_','post_'])
    df_sum['diff'] = df_sum['pre_fruits_num'] - df_sum['post_fruits_num']
    df_sum.to_csv(output,index=True,header=True,mode='w')
    return df_pre, df_post

In [83]:
scan_pre = fr'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\tomato\230123\pre'
scan_post = fr'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\tomato\230123\post'
output_path = fr'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\tomato\230123\results.csv'
df_pre, df_post = run_counter(scan_pre,scan_post,output_path)

1 1
        x1    y1    x2    y2  obj_conf  class_conf  track_id  frame  cluster  \
0      937   478  1034   571  0.911096         0.0         5     10        0   
1      904   375  1006   487  0.902273         0.0        11     10        0   
2      990   308  1080   421  0.890364         0.0        15     10        0   
3      928   909  1035  1014  0.885934         0.0        18     10        1   
4     1011   421  1080   508  0.883765         0.0        19     10        0   
...    ...   ...   ...   ...       ...         ...       ...    ...      ...   
9168   344  1116   409  1176  0.823383         0.0      2100    551        3   
9169   130   663   203   750  0.844912         0.0      2102    551        0   
9170     0    74    37   158  0.858765         0.0      2077    552        0   
9171   263   378   324   454  0.861253         0.0      2091    552        1   
9172   339   600   402   669  0.884002         0.0      2101    552        2   

         height      width  distanc

KeyError: 'tree'

Debug size

In [ ]:
def apply_trees_size(path,row):
    row_folder = os.path.join(path,row)
    df_res = filter_logic(row_folder,analysis='size')
    trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]),None,None)
    for tree_id, df_tree in trees_slices.groupby('tree_id'):
        measures = trackers_into_plot(df_tree,df_res,analysis='size')
        count = trackers_into_value(df_tree, df_res, analysis='counter')
        yield tree_id, count, measures
        plot_id = map_tree_into_plot(row, tree_id)
        df_sum = df_sum.append({'repeat': type, 'row': row, 'plot_id': plot_id, 'fruits_num': count}, ignore_index=True)
    return df_sum


    return count,measures

In [ ]:
def run_size(scan_pre, scan_post, output):
    df_sum = pd.DataFrame()
    for (pre, post) in zip(os.listdir(scan_pre), os.listdir(scan_post)):
        for (pre_tree_id, pre_count, pre_measures), (post_tree_id,post_count, post_measures) in zip(apply_trees_size(scan_pre, pre), apply_trees_size(scan_post, post)):
            mean, std = diff_size(1 - pre_count / post_count, pre_measures, post_measures)
            plot_id = map_tree_into_plot(pre,pre_tree_id)
            df_sum = df_sum.append({'row': row, 'plot_id': plot_id, 'avg_size': mean,'std_size':std}, ignore_index=True)

In [ ]:
scan_pre = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222'
scan_post = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222'
output_path = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222/results.csv'
run_size(scan_pre,scan_post,output_path)

Debug Color

In [ ]:
def run_color():
    for index, row in enumerate(os.listdir(scan)):
        row_folder = os.path.join(scan,row)
        if not row.__contains__('TiltDown40cm') or not row.__contains__('2nd'):
            continue
        df_res = filter_logic(row_folder,analysis='color')
        trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]),None,None)
        color_df = trackers_into_plot(trees_slices,df_res,analysis='color')
        count = trackers_into_plot(trees_slices,df_res,analysis='counter')
        if row.__contains__('pre'):
            plt.hist(size_df)
            pre_count = count
            pre_measures = color_df
        if row.__contains__('post'):
            plt.hist(size_df)
            post_count =count
            post_measures = color_df
        plt.title(row)
        plt.show()

    print(diff_size(1-post_count/pre_count, all_measures= pre_measures,nonPicked_measures=post_measures))



In [ ]:
df = pd.read_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\061022\color_for_hist_2676.csv')
df['color_median'] = None
df['color_mean'] = None
for j, row in df.iterrows():
    try:
        hue_med,hue_mean = get_color(plot_bbox(row, (0, j)), index=row['frame'])
        df.at[j, 'color_median'] = hue_med
        df.at[j, 'color_mean'] = hue_mean
    except Exception as e:
        print(repr(e))
df.to_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\061022\color_for_hist_2676.csv', index=False)